In [3]:
import os

In [4]:
%pwd

'c:\\Users\\RaviB\\GitHub\\SleepEfficiencyML\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\RaviB\\GitHub\\SleepEfficiencyML'

In [7]:
import pandas as pd

In [8]:
data = pd.read_csv("artifacts/data_ingestion/Sleep_Efficiency.csv")
data.head()

,ID,Age,Gender,Bedtime,Wakeup time,Sleep duration,Sleep efficiency,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Alcohol consumption,Smoking status,Exercise frequency
0,1,65,Female,2021-03-06 01:00:00,2021-03-06 07:00:00,6.0,0.88,18,70,12,0.0,0.0,0.0,Yes,3.0
1,2,69,Male,2021-12-05 02:00:00,2021-12-05 09:00:00,7.0,0.66,19,28,53,3.0,0.0,3.0,Yes,3.0
2,3,40,Female,2021-05-25 21:30:00,2021-05-25 05:30:00,8.0,0.89,20,70,10,1.0,0.0,0.0,No,3.0
3,4,40,Female,2021-11-03 02:30:00,2021-11-03 08:30:00,6.0,0.51,23,25,52,3.0,50.0,5.0,Yes,1.0
4,5,57,Male,2021-03-13 01:00:00,2021-03-13 09:00:00,8.0,0.76,27,55,18,3.0,0.0,3.0,No,3.0


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      452 non-null    int64  
 1   Age                     452 non-null    int64  
 2   Gender                  452 non-null    object 
 3   Bedtime                 452 non-null    object 
 4   Wakeup time             452 non-null    object 
 5   Sleep duration          452 non-null    float64
 6   Sleep efficiency        452 non-null    float64
 7   REM sleep percentage    452 non-null    int64  
 8   Deep sleep percentage   452 non-null    int64  
 9   Light sleep percentage  452 non-null    int64  
 10  Awakenings              432 non-null    float64
 11  Caffeine consumption    427 non-null    float64
 12  Alcohol consumption     438 non-null    float64
 13  Smoking status          452 non-null    object 
 14  Exercise frequency      446 non-null    fl

Let's inspect and clean the data a bit.

In [10]:
data.isnull().sum()

ID                         0
Age                        0
Gender                     0
Bedtime                    0
Wakeup time                0
Sleep duration             0
Sleep efficiency           0
REM sleep percentage       0
Deep sleep percentage      0
Light sleep percentage     0
Awakenings                20
Caffeine consumption      25
Alcohol consumption       14
Smoking status             0
Exercise frequency         6
dtype: int64

In [14]:
from collections import Counter

In [22]:
print(Counter(data["Awakenings"]))

Counter({1.0: 154, 0.0: 95, 3.0: 63, 4.0: 63, 2.0: 57, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1})


In [20]:
print(Counter(data["Caffeine consumption"]))

Counter({0.0: 211, 50.0: 107, 25.0: 79, 75.0: 25, 200.0: 4, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, 100.0: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1})


In [18]:
print(Counter(data["Alcohol consumption"]))

Counter({0.0: 246, 1.0: 54, 3.0: 48, 2.0: 37, 5.0: 30, 4.0: 23, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1})


In [23]:
print(Counter(data["Exercise frequency"]))

Counter({3.0: 130, 0.0: 116, 1.0: 97, 2.0: 54, 4.0: 41, 5.0: 8, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1, nan: 1})


Since we don't have a lot of data, we will just fill missing values with the most frequent value.

Also, we will modify Caffeine and Alcohol consumption columns to be binary (so 0 for no consumption and 1 for at least some).

In [24]:
data_filled = data.apply(lambda x: x.fillna(x.mode()[0]) if x.isnull().any() else x)

In [26]:
# just double check one of the columns to make sure it worked
print(Counter(data_filled["Exercise frequency"]))

Counter({3.0: 136, 0.0: 116, 1.0: 97, 2.0: 54, 4.0: 41, 5.0: 8})


In [25]:
data_filled.isnull().sum()

ID                        0
Age                       0
Gender                    0
Bedtime                   0
Wakeup time               0
Sleep duration            0
Sleep efficiency          0
REM sleep percentage      0
Deep sleep percentage     0
Light sleep percentage    0
Awakenings                0
Caffeine consumption      0
Alcohol consumption       0
Smoking status            0
Exercise frequency        0
dtype: int64

Now let's build the data class.

In [69]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [70]:
from sleep_efficiency.constants import *
from sleep_efficiency.utils.common import read_yaml, create_directories


In [71]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
    
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE = config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema = schema
        )

        return data_validation_config

In [72]:
import os
from sleep_efficiency import logger

In [73]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}") 
            return validation_status
        except Exception as e:
            raise e



In [74]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-06-25 10:13:55,188: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-25 10:13:55,190: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-25 10:13:55,195: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-25 10:13:55,195: INFO: common: created directory at artifacts]
[2024-06-25 10:13:55,196: INFO: common: created directory at artifacts/data_validation]
